<a href="https://colab.research.google.com/github/kentstephen/duckdb_h3/blob/main/fsq_within_overture_budilings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Determine Foursquare POIs Inside of Overture Buildings with Fused, DuckDB and H3

In [ ]:
! pip install duckdb pandas geopandas shapely fused affine pydeck h3 -q

In [ ]:
import fused
import pandas as pd
import geopandas as gpd
import shapely
import h3
import pydeck as pdk

I used this [bounding box generator](https://boundingbox.klokantech.com/) to find the bounds for the French Quarter in New Orleans. If you want to try a different AOI: select CSV and paste below.

In [ ]:
bounds = gpd.GeoDataFrame(
    geometry=[shapely.box(-90.093538,29.938505,-90.043702,29.984352)], # New Orleans, French Quarter
    crs=4326
)

This uses the [Fused.io](https://fused.io) UDF to obtain the Foursquare data from New Orleans. We create lat and lng columns for H3 and preserve the geometry as WKT.

In [ ]:
def get_fsq(bounds):
    gdf = fused.run("UDF_Foursquare_Open_Source_Places", bounds=bounds, min_zoom=0)
    if gdf is None or gdf.empty:
        return pd.DataFrame({})
    if gdf.crs != "EPSG:4326":
          gdf.to_crs("EPSG:4326")
    gdf['lat'] = gdf.geometry.y
    gdf['lng'] = gdf.geometry.x
    gdf['geometry'] = gdf['geometry'].apply(shapely.wkt.dumps)
    return pd.DataFrame(gdf)
df_fsq = get_fsq(bounds)
print(df_fsq.head())

           fsq_category_ids                                          geometry  \
0  4bf58dd8d48988d100941735  POINT (-90.0677140011325719 29.9525342796529017)   
1  4bf58dd8d48988d100941735  POINT (-90.0673611595095167 29.9529434436182527)   
2  4bf58dd8d48988d100941735  POINT (-90.0626904000000081 29.9475211500000000)   
3  4bf58dd8d48988d100941735  POINT (-90.0646069999999952 29.9394689999999990)   
4  4bf58dd8d48988d100941735  POINT (-90.0673754272572893 29.9528779291416996)   

                                   name                level1_category_name  \
0       Galerie 4 - New Orleans Marriot  Business and Professional Services   
1  Carondelet Room- New Orleans Marriot  Business and Professional Services   
2               ACUHO-I Opening Session  Business and Professional Services   
3      Microsoft TechEd 2010 PASS Booth  Business and Professional Services   
4              NRECA Connect Conference  Business and Professional Services   

  level2_category_name level3_category

<ipython-input-33-567a3a56f64c>:9: UserWarning: Geometry column does not contain geometry.
  gdf['geometry'] = gdf['geometry'].apply(shapely.wkt.dumps)


Same for Overture Buildings.

In [ ]:
def get_buildings(bounds, overture_type):
    overture_utils = fused.load("https://github.com/fusedio/udfs/tree/ee9bec5/public/Overture_Maps_Example/").utils # Load pinned versions of utility functions.
    gdf = overture_utils.get_overture(bbox=bounds, overture_type=overture_type, min_zoom=0)
    if gdf is None or gdf.empty:
        return pd.DataFrame({})
    if gdf.crs != "EPSG:4326":
          gdf.to_crs("EPSG:4326")
    gdf['geometry'] = gdf['geometry'].apply(shapely.wkt.dumps)
    return pd.DataFrame(gdf)
df_buildings = get_buildings(bounds=bounds, overture_type="building")
print(df_buildings.head())

2025-03-09 22:00:40.475 | DEBUG    | fused.core._cache:_read_cache_log:237 - Recent cache entry found in /tmp/cached_data/table_to_tile/data_9f836d0516619504bd3b8e716db7a572b15d8c9759c62392bad4e657747d107e/index.log: '2025-03-10T08:57:50.414914+00:00', 'ff00bc5b-fcec-44f1-99b8-65189dede0ed', 'd'
2025-03-09 22:00:40.477 | DEBUG    | fused.core._cache:_cache:338 - Detected status is d and expiration is 2025-03-10 08:57:50.414914+00:00
2025-03-09 22:00:40.481 | DEBUG    | fused.core._cache:_cache:342 - Reading cache for table_to_tile under ff00bc5b-fcec-44f1-99b8-65189dede0ed
2025-03-09 22:00:40.492 | DEBUG    | fused.core._cache:_read_cache_log:237 - Recent cache entry found in /tmp/cached_data/table_to_tile/data_10b2123e07390fb104f87801035921933e9ac0a904c197e6384f3b3d55e90251/index.log: '2025-03-10T08:57:50.418333+00:00', '4bf52bb1-2f22-4932-9fe7-fab5e41b9820', 'd'
2025-03-09 22:00:40.505 | DEBUG    | fused.core._cache:_cache:338 - Detected status is d and expiration is 2025-03-10 08:57

                                    id  \
1880  08b4446425010fff0200ec116754b5ad   
1881  08b4446425011fff0200415569d27fc5   
1882  08b4446425013fff0200ff1be4e6f27d   
1883  08b444642518bfff020044394b43c546   
1884  08b44464250a5fff020090075cfb62f5   

                                               geometry  \
1880  POLYGON ((-90.0813514000000026 29.945266700000...   
1881  POLYGON ((-90.0816563000000059 29.945435499999...   
1882  POLYGON ((-90.0810781319999592 29.946035517000...   
1883  POLYGON ((-90.0809997000000067 29.944935399999...   
1884  POLYGON ((-90.0804794000000015 29.945232399999...   

                                                   bbox  version  \
1880  {'xmin': -90.08163452148438, 'xmax': -90.08125...        0   
1881  {'xmin': -90.0817642211914, 'xmax': -90.081550...        0   
1882  {'xmin': -90.08155059814453, 'xmax': -90.08065...        0   
1883  {'xmin': -90.08100128173828, 'xmax': -90.08010...        0   
1884  {'xmin': -90.08077239990234, 'xmax': -90.08046

<ipython-input-34-8b9b79378c6e>:8: UserWarning: Geometry column does not contain geometry.
  gdf['geometry'] = gdf['geometry'].apply(shapely.wkt.dumps)


We need to use a constant resolution for the H3 transformations, I suggest between 13 and 15 for this exercise. Although if you don't mind less precision, lowering the resolution would give you a buffer around buildings.

In [ ]:
resolution = 13

Here we use DuckDB with the necessary extensions loaded with Fused common utilities. Since `h3_polygon_wkt_to_cells` returns an array, we need to use a CTE. At the end we are grouping by the Overture buildings GERS ID.

There is a new DuckDB function called `h3_polygon_wkt_to_cells_experimental` with an updated algorythm but I have not had success with it yet.

In [ ]:
def polyfill_buildings(df, resolution):
    utils = fused.load("https://github.com/fusedio/udfs/tree/ee9bec5/public/common/").utils
    # Connect to DuckDB
    con = utils.duckdb_connect()
    query=f"""
    with buildings_to_cells as (
        select
            id,
            unnest(h3_polygon_wkt_to_cells(geometry, {resolution})) as hex
        from df
    )
    select
        id,
        hex
    from buildings_to_cells b
    group by 1, 2
    """
    return con.sql(query).df()

df_buildings_h3 = polyfill_buildings(df=df_buildings, resolution=resolution)
print(df_buildings_h3.head())

duckdb version: 1.1.3


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

                                 id                 hex
0  08b4446425010fff0200ec116754b5ad  636208644274326207
1  08b4446425010fff0200ec116754b5ad  636208644274326463
2  08b4446425010fff0200ec116754b5ad  636208644274326143
3  08b4446425010fff0200ec116754b5ad  636208644274323839
4  08b4446425010fff0200ec116754b5ad  636208644274326271


Now that we have Overture buildings as cells with IDs we can join with the Foursquare places using `h3_latlng_to_cell` in the DuckDB join condition to filter for points inside the buildings using the same H3 resolution.

In [ ]:
def join_h3_buildings_with_fsq(df_buildings_h3, df_fsq, resolution):
    # Load pinned versions of utility functions.
    utils = fused.load("https://github.com/fusedio/udfs/tree/ee9bec5/public/common/").utils
    # Connect to DuckDB
    con = utils.duckdb_connect()

    query = f"""
        select
          b.id as overture_building_id,
          f.name,
          f.geometry as geometry_wkt,
          f.level1_category_name,
          f.level2_category_name,
          f.level3_category_name,
          f.fsq_category_ids
        from df_buildings_h3 b inner join df_fsq f
        on b.hex = h3_latlng_to_cell(f.lat, f.lng, {resolution})
    """

    # Run the query and return a GeoDataFrame
    df = con.sql(query).df()
    return gpd.GeoDataFrame(df.drop(columns=['geometry_wkt']), geometry=df.geometry_wkt.apply(shapely.wkt.loads))

gdf_fsq_in_buildings = join_h3_buildings_with_fsq(df_buildings_h3, df_fsq, resolution)
print(gdf_fsq_in_buildings.head())

duckdb version: 1.1.3
               overture_building_id                                 name  \
0  08b4446420a9dfff020071ad1dd9b831  French Quarter Business Association   
1  08b4446420a9dfff020071ad1dd9b831                    French Connection   
2  08b4446420a9dfff020071ad1dd9b831                        Vip city tour   
3  08b4446420a9dfff020071ad1dd9b831                        Urban Systems   
4  08b4446420a9dfff020071ad1dd9b831                                  H&M   

       level1_category_name level2_category_name     level3_category_name  \
0  Community and Government         Organization  Non-Profit Organization   
1                    Retail       Fashion Retail            Women's Store   
2                      None                 None                     None   
3    Landmarks and Outdoors            Structure                     None   
4                    Retail       Fashion Retail            Women's Store   

           fsq_category_ids                    geometry  


As you can see above, now every POI has a building ID.

In order to visualize with pydeck we need to convert the point geometry.

In [ ]:
gdf_fsq_in_buildings["pydeck_geometry"] = [[c.x, c.y] for c in gdf_fsq_in_buildings["geometry"]]

Further prep for pydeck involves converting the H3 integer (which is more efficient for the transformations) to the string.

In [ ]:
# pydeck needs the H3 string so we convert
df_buildings_h3['hex'] = df_buildings_h3['hex'].apply(h3.int_to_str)

This vizualizes the building cells and POIs within them.

In [ ]:
#Layer for FSQ Places we joined wit H3 Overture buildings
fsq_joined_layer = pdk.Layer(
    "ScatterplotLayer",
    gdf_fsq_in_buildings,
    pickable=True,
    stroked=False,
    filled=True,
    line_width_min_pixels=1,
    get_position="pydeck_geometry",
    get_fill_color=[255, 140, 0],
    radius_min_pixels=3,
    opacity=0.8,
)

# Layer for the polyfilled buildings
building_h3_layer = pdk.Layer(
    "H3HexagonLayer",
    data=df_buildings_h3,
    get_hexagon="hex",
    stroked=False,
    get_fill_color=[0, 255, 255],
    coverage=1,
    extruded=False,
    pickable=True,
    opacity=0.3
)

# Set view state
view_state = pdk.ViewState(
    latitude=df_fsq["lat"].mean(),
    longitude=df_fsq["lng"].mean(),
    zoom=14,
)

# Create the map
r = pdk.Deck(layers=[building_h3_layer, fsq_joined_layer], initial_view_state=view_state)
r.show()

## Let me know what you think!

Find me on [LinkedIn](https://www.linkedin.com/in/stephen-kent-37b97454)